# Tutorial part 1
## Neuron models

In [1]:
from brian2 import *
%matplotlib widget
prefs.codegen.target = 'numpy'  # Examples are too simple for code generation

### Leaky integrate-and-fire model

In [ ]:
start_scope()  # because we run several examples in this notebook

N = 1000
duration = 1*second
tau = 10*ms
eqs = '''
dv/dt = (v0 - v) / tau : volt (unless refractory)
v0 : volt
'''
group = NeuronGroup(N, eqs, threshold='v > 10*mV', reset='v = 0*mV',
                    refractory=1*ms, method='exact')
group.v = 0*mV
group.v0 = '20*mV* i / (N-1)'

monitor = SpikeMonitor(group)

run(duration)

In [ ]:
# Usually a good idea to split off plotting/analysis code from simulation
plt.figure()
plt.plot(group.v0/mV, monitor.count / duration, 'o-')
plt.xlabel('v0 (mV)')
plt.ylabel('Firing rate (sp/s)');

### Hodgkin-Huxley model

In [ ]:
start_scope()

area = 20000*umetre**2
Cm = 1*ufarad*cm**-2 * area
gl = 5e-5*siemens*cm**-2 * area
El = -65*mV
EK = -90*mV
ENa = 50*mV
g_na = 100*msiemens*cm**-2 * area
g_kd = 30*msiemens*cm**-2 * area
VT = -63*mV

# The model
eqs = Equations('''
dv/dt = (gl*(El-v) - g_na*(m*m*m)*h*(v-ENa) - g_kd*(n*n*n*n)*(v-EK) + I)/Cm : volt
dm/dt = 0.32*(mV**-1)*4*mV/exprel((13.*mV-v+VT)/(4*mV))/ms*(1-m)-0.28*(mV**-1)*5*mV/exprel((v-VT-40.*mV)/(5*mV))/ms*m : 1
dn/dt = 0.032*(mV**-1)*5*mV/exprel((15.*mV-v+VT)/(5*mV))/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
I : amp
''')
# Threshold and refractoriness are only used for spike counting
group = NeuronGroup(N, eqs,
                    threshold='v > -40*mV',
                    refractory='v > -40*mV',
                    method='exponential_euler')
group.v = El
group.I = '0.7*nA * i / N'

monitor = SpikeMonitor(group)
v_mon = StateMonitor(group, 'v', record=True)

run(duration, report='text')

In [ ]:
plt.figure()
plt.plot(group.I/nA, monitor.count / duration, 'o-')
plt.xlabel('I (nA)')
plt.ylabel('Firing rate (sp/s)');

In [ ]:
plt.figure()
for idx in [50, 500]:
    plt.plot(v_mon.t/ms, v_mon.v[idx]/mV, label=f'I={group.I[idx]/nA:.2f}nA', alpha=0.5)
plt.legend()
plt.xlabel('time (ms)')
plt.ylabel('v (mV)');

## Synapse models

### Randomly connected network of LIF neurons

In [ ]:
start_scope()
taum = 20*ms
taue = 5*ms
taui = 10*ms
Vt = -50*mV
Vr = -60*mV
El = -49*mV

eqs = '''
dv/dt  = (ge+gi-(v-El))/taum : volt (unless refractory)
dge/dt = -ge/taue : volt
dgi/dt = -gi/taui : volt
'''

P = NeuronGroup(4000, eqs, threshold='v>Vt', reset='v = Vr', refractory=5*ms,
                method='exact')
P.v = 'Vr + rand() * (Vt - Vr)'
P.ge = 0*mV
P.gi = 0*mV

we = (60*0.27/10)*mV # excitatory synaptic weight (voltage)
wi = (-20*4.5/10)*mV # inhibitory synaptic weight
Ce = Synapses(P, P, on_pre='ge += we')
Ci = Synapses(P, P, on_pre='gi += wi')
Ce.connect('i<3200', p=0.02)
Ci.connect('i>=3200', p=0.02)

s_mon = SpikeMonitor(P)

run(1 * second)

In [ ]:
plt.figure()
plt.plot(s_mon.t/ms, s_mon.i, ',k')
plt.xlabel('Time (ms)')
plt.ylabel('Neuron index');

### Connection patterns

In [ ]:
def visualise_connectivity(S):
    Ns = len(S.source)
    Nt = len(S.target)
    plt.figure(figsize=(8, 4))
    plt.subplot(121)
    plt.plot(zeros(Ns), arange(Ns), 'ok', ms=10)
    plt.plot(ones(Nt), arange(Nt), 'ok', ms=10)
    for i, j in zip(S.i, S.j):
        plt.plot([0, 1], [i, j], '-k')
    plt.xticks([0, 1], ['Source', 'Target'])
    plt.ylabel('Neuron index')
    plt.xlim(-0.1, 1.1)
    plt.ylim(-1, max(Ns, Nt))
    plt.subplot(122)
    plt.plot(S.i, S.j, 'ok')
    plt.xlim(-1, Ns)
    plt.ylim(-1, Nt)
    plt.xlabel('Source neuron index')
    plt.ylabel('Target neuron index')

In [ ]:
# Dummy group
N = 10
G = NeuronGroup(N, 'v:1')

In [ ]:
# Random connectivity + condition
S = Synapses(G, G)
S.connect('i != j', p=0.5)
visualise_connectivity(S)

In [ ]:
# More complex condition
S = Synapses(G, G)
S.connect('abs(i - j) < 3 and i != j')
visualise_connectivity(S)

In [ ]:
# More efficient with "generator syntax": how to calculate targets for each source cell
S = Synapses(G, G)
S.connect(j='k for k in range(i-3, i+4) if i!=k', skip_if_invalid=True)
visualise_connectivity(S)

In [ ]:
# Fixed number of targets for each sample
S = Synapses(G, G)
S.connect(j='k for k in sample(0, N_post, size=3)')
visualise_connectivity(S)

In [ ]:
# Calculated probability
S = Synapses(G, G)
S.connect('i != j', p='exp(-abs(i - j)/2)')
visualise_connectivity(S)

### Spike-timing dependent plasticity

In [ ]:
start_scope()

N = 1000
taum = 10*ms; taupre = 20*ms; taupost = taupre
Ee = 0*mV; vt = -54*mV; vr = -60*mV; El = -74*mV; taue = 5*ms
F = 15*Hz; gmax = .01
dApre = .01; dApost = -dApre * taupre / taupost * 1.05; dApost *= gmax; dApre *= gmax

eqs_neurons = '''
dv/dt = (ge * (Ee-v) + El - v) / taum : volt
dge/dt = -ge / taue : 1
'''

poisson_input = PoissonGroup(N, rates=F)
neurons = NeuronGroup(1, eqs_neurons, threshold='v>vt', reset='v = vr',
                      method='euler')
S = Synapses(poisson_input, neurons,
             '''w : 1
                dApre/dt = -Apre / taupre : 1 (event-driven)
                dApost/dt = -Apost / taupost : 1 (event-driven)''',
             on_pre='''ge += w
                    Apre += dApre
                    w = clip(w + Apost, 0, gmax)''',
             on_post='''Apost += dApost
                     w = clip(w + Apre, 0, gmax)''',
             )
S.connect()
S.w = 'rand() * gmax'
mon = StateMonitor(S, 'w', record=[0, 1])
s_mon = SpikeMonitor(poisson_input)

run(10*second, report='text')

In [ ]:
plt.figure()
plt.subplot(311)
plt.plot(S.w / gmax, '.k')
plt.ylabel('Weight / gmax')
plt.xlabel('Synapse index')
plt.subplot(312)
plt.hist(S.w / gmax, 20)
plt.xlabel('Weight / gmax')
plt.subplot(313)
plt.plot(mon.t/second, mon.w.T/gmax)
plt.xlabel('Time (s)')
plt.ylabel('Weight / gmax')
plt.tight_layout();